In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import time

start = time.time()

# Process Data

## Read

In [2]:
df = pd.read_csv('raw.csv', index_col=0)

## Process

In [3]:
rows = df.shape[0]
print(f'Shape before preprocessing: {df.shape}')

# Must have wage data, either '76 or '78
valid = np.argwhere(~df[['lwage76', 'lwage78']].isnull().all(axis=1).values).ravel()
df = df.iloc[valid, :].reset_index(drop=True)
# Must have values for these features
df = df.dropna(subset=['smsa78r', 'reg78r', 'enroll78', 'marsta78']).reset_index()

# Impute IQ & KWW
df['imputeiq'] = df['iq'].isnull()*1
df['iq'] = df['iq'].fillna(df['iq'].mean())
df['imputekww'] = df['kww'].isnull()*1
df['kww'] = df['kww'].fillna(df['kww'].mean())

# If libcrd14 is null, impute 0
df['libcrd14'] = df['libcrd14'].fillna(0)
print(f'Shape after preprocessing: {df.shape}')
print(f'Data loss: {100*(1-df.shape[0]/rows):.2f}%')

Shape before preprocessing: (3613, 52)
Shape after preprocessing: (2993, 55)
Data loss: 17.16%


## Get Relevant Variables

In [4]:
# Instrumental variable is proximity to 2-year or 4-year college
iv = df['nearc4'].values

# Except for these, we use '78 data
# Education in '76
treatment = df['ed76'].values

# Geometric mean of '76 and '78 log-wage
result = np.exp(np.mean(np.log(df[['lwage76', 'lwage78']]), axis=1)).values

covariates = df.drop(columns=['id', 'nearc2', 'nearc4', 'nearc4a', 'nearc4b', 'ed66', 'ed76', 'lwage76',
                              'lwage78', 'work76', 'work78', 'reg80r', 'wage76', 'wage78', 'wage80',
                              'noint80', 'enroll80', 'marsta76', 'marsta80', 'smsa66r', 'smsa76r',
                              'enroll76', 'reg76r'])

# Analysis: Two-Stage Least Squares

## LSE of IV Effect to Treatment Value

In [5]:
# Mean as unbiased estimator of binary IV effect
estimate = {0: treatment[iv==0].mean(), 1: treatment[iv==1].mean()}

# Map from true IV value
estimates = [estimate[x] for x in iv]

## LSE of Estimated Treatment Value to Response Variable

In [6]:
# Incorporate estimates
x = covariates.copy()
x['estimates'] = estimates

lr = LinearRegression()
lr.fit(x, result)
prediction = lr.predict(x)

## Calculate Effect

In [7]:
effect = prediction[iv==1].mean() - prediction[iv==0].mean()

# Direction of effect
sign = {True: 'increases', False: 'decreases'}

print(f'Result:\nProximity {sign[effect > 0]} wage by {100*(np.exp(effect)-1):.2f}%')

Result:
Proximity increases wage by 15.35%


In [8]:
print(f'Elapsed: {time.time()-start:.2f}s')

Elapsed: 0.11s
